# Plotting Helper

In [ ]:
def plot_coverage_comparison(
    df_uncal,
    df_cal,
    *,
    alpha_col: str = "alpha",
    cov_col: str = "coverage",
    labels=("Before CP", "After CP"),
    title: str = "Coverage Comparison",
    ax=None,
    # --- NEW style controls to align with field plots ---
    figsize=(6, 5),          # match plot_uq_field default
    n_ticks: int = 9,        # number of major ticks on each axis (incl. endpoints)
    font_size: int = 12,     # label/title font
    tick_size: int = 10,     # tick label size
    grid_alpha: float = 0.2, # grid transparency
    box_aspect: float = 1.0, # make axes square like your field panels
    # --- extras to match your latest styling ---
    lw: float = 5.0,         # line width for both curves
    ms1: float = 13.0,       # marker size for uncal curve
    ms2: float = 20.0,       # marker size for cal curve
    pad: float = 0.03,       # extra margin so endpoints are visible
    legend_fontsize: int = 14
):
    """
    Plot empirical vs expected coverage for two models, styled to align with the 2D field plots.
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.ticker import LinearLocator, FormatStrFormatter

    def _prepare(df):
        exp = 1.0 - df[alpha_col].to_numpy()
        emp = df[cov_col].to_numpy()
        # include endpoints for cleaner lines
        exp = np.concatenate(([0.0], exp, [1.0]))
        emp = np.concatenate(([0.0], emp, [1.0]))
        idx = np.argsort(exp)
        return exp[idx], emp[idx]

    exp1, emp1 = _prepare(df_uncal)
    exp2, emp2 = _prepare(df_cal)

    if ax is None:
        fig, ax = plt.subplots(figsize=figsize)
    else:
        fig = ax.figure

    # Curves + ideal diagonal
    ax.plot(exp1, emp1, marker='o', linestyle='-', color="#b13c32",
            markerfacecolor="#ed8076", ms=ms1, lw=lw, label=f'{labels[0]}',
            clip_on=False)
    ax.plot(exp2, emp2, marker='*', linestyle='-', color="#3245b1",
            markerfacecolor="#6988ef", ms=ms2, lw=lw, label=f'{labels[1]}',
            clip_on=False)
    ax.plot([0, 1], [0, 1], '--', color='gray', lw=max(2, lw*0.4), label='Ideal (y=x)',
            clip_on=False)

    # Limits, ticks, grid to match the field panels
    # Use autoscale + margins so endpoints aren't cramped
    ax.autoscale(enable=True, tight=False)
    ax.margins(x=pad, y=pad)

    ax.xaxis.set_major_locator(LinearLocator(n_ticks))
    ax.yaxis.set_major_locator(LinearLocator(n_ticks))
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

    # Fonts & sizing consistent with field plots
    ax.set_xlabel("Expected Coverage (1 − α)", fontsize=font_size)
    ax.set_ylabel("Empirical Coverage", fontsize=font_size)
    ax.set_title(title, fontsize=font_size)
    ax.tick_params(axis='both', labelsize=tick_size)

    # Make the plotting box square (like the imshow panels)
    try:
        ax.set_box_aspect(box_aspect)  # mpl>=3.3
    except Exception:
        pass

    # Subtle grid
    ax.grid(True, alpha=grid_alpha)

    ax.legend(loc='lower right', fontsize=legend_fontsize)
    fig.tight_layout()
    return ax


# Data

In [ ]:
# Basic
import torch
import math
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import numpy as np
from numpy import random
torch.set_num_threads(4)

# PDEi
from utils_pde.utils_pde_2dpoisson import Poisson2D

# Viz
from utils_tools.utils_result_viz import plot_predictions_2D

# Base Mdoels
from utils_uqmd.utils_uq_dropout import DropoutPINN
from utils_uqmd.utils_uq_mlp import MLPPINN
from utils_uqmd.utils_uq_vi import VIBPINN
from utils_uqmd.utils_uq_distance import DistanceUQPINN

# CP
from utils_uqmd.utils_uq_cp import CP

# Ensure reproducibility
import random, numpy as np, torch
seed = 456 # 12345
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# ----------------------------------------------------------------------
# Data Noise
data_noise = 0.05          # same as your 2-D example

# Alpha
alpha = 0.05
# Generating alphas to test
from utils_tools.utils_result_metrics import generating_alphas
alphas = generating_alphas()


# Define the 3-D Helmholtz PDE
from utils_pde.utils_pde_3dhelmholtz import Helmholtz3D
domain = ((0.0, 1.0), (0.0, 1.0), (0.0, 1.0))   # (x0,x1),(y0,y1),(z0,z1)
k = math.pi                                     # wave number

true_solution = (
    lambda xyz: torch.sin(math.pi * xyz[..., 0:1])
              * torch.sin(math.pi * xyz[..., 1:2])
              * torch.sin(math.pi * xyz[..., 2:3])
)

pde = Helmholtz3D(k=k, domain=domain, true_solution=true_solution)

# Generate training / testing / calibration data
(X_train, Y_train)         = pde.data_generation(600, data_noise)
(X_test,  Y_test)          = pde.data_generation(200, data_noise)
(X_calibration, Y_calibration) = pde.data_generation(200, data_noise)


# Not used in experiment, only use to generate different file name
(X_validation, Y_validation) = pde.data_generation(400, data_noise)

# Number of interior collocation points for the PINN residual
colloc_pt_num = 8000

# --------------------------------------------
# (not important, keeped for function call compatability)
# --------------------------------------------
n_grid = 10 
x = torch.linspace(domain[0][0], domain[0][1], n_grid)
y = torch.linspace(domain[1][0], domain[1][1], n_grid)
z = torch.linspace(domain[2][0], domain[2][1], n_grid)
X, Y, Z = torch.meshgrid(x, y, z, indexing='xy')
grid_test = torch.cat([X.reshape(-1, 1), Y.reshape(-1, 1), Z.reshape(-1, 1)], dim=1)


# Dropout Model

## Define Model

In [ ]:
from utils_uqmd.utils_uq_dropout import DropoutPINN

# Base Model Instance
model_args = dict(
    pde_class=pde, 
    input_dim=3,
    hidden_dims=[32, 64, 128, 128, 128, 64, 32], 
    output_dim=1
)
baseline_pred_kwargs = {  
    "n_samples":40
}

# CP Model
cp_testing_args = {
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test, 
    "X_cal":X_calibration, "Y_cal":Y_calibration, 
    "X_train":X_train, "Y_train":Y_train, 
    "heuristic_u":"raw_std", # Change base on if the baseline cp
    "k":100
}

cp_pred_kwargs = {
    "X_train":X_train,  "Y_train":Y_train,
    "X_cal":X_calibration, "Y_cal":Y_calibration,
    "heuristic_u":"raw_std",
    "k":100
}

baseline_testing_args = { 
    # "uqmodel":do_pinn,   # Change this
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test,
    "n_samples":40
}

dropout_pinn = DropoutPINN(**model_args)

## Training Base Model

In [ ]:
# Base Model
# Training
fit_args = dict(
    coloc_pt_num=colloc_pt_num, 
    X_train=X_train, 
    Y_train=Y_train
)
fit_kwargs_grid = dict(
    epochs=5000,
    λ_pde=1.0, λ_bc=10.0, λ_data=1.0,
    lr=1e-3, stop_schedule=20000
)
baseline_loss_dict = dropout_pinn.fit(**fit_args, **fit_kwargs_grid)

# Test the model performance
# baseline_data_loss = dropout_pinn.data_loss(X_validation, Y_validation)


# Predicting
baseline_pred_kwargs = dict(
    n_samples=40
)
dropout_pinn_uncal_predset = dropout_pinn.predict(alpha, grid_test, 
                                   **baseline_pred_kwargs)


## Create CP Model

In [ ]:
# CP Model
dropout_pinn_cp = CP(dropout_pinn)

# Predicting
cp_pred_kwargs = {
        "X_train":X_train,  "Y_train":Y_train,
        "X_cal":X_calibration, "Y_cal":Y_calibration,
        "heuristic_u":"raw_std",  # Change this based on cp
        "k":100
}

dropout_pinn_cp_cal_predset = dropout_pinn_cp.predict(
        alpha=alpha, X_test=grid_test,
        **cp_pred_kwargs
)

## Plot Coverage Plot

In [ ]:
# Computing Coverage Info
from utils_tools.utils_result_metrics import cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties

# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_metrics_table, plot_dual_expected_vs_empirical
dropout_pinn_df_uncal = do_test_uncertainties(uqmodel=dropout_pinn, **baseline_testing_args)
dropout_pinn_df_cal = cp_test_uncertainties(dropout_pinn_cp, **cp_testing_args)

# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_metrics_table
save_plot(
    plot_metrics_table,
    save_dir="3D_Helmholtz", prefix="dropout",
    # loss=baseline_data_loss
)(
    grid_test, dropout_pinn_uncal_predset, dropout_pinn_cp_cal_predset, 
    true_solution, 
    dropout_pinn_df_uncal, dropout_pinn_df_cal,
    title="2D UQ", main_title="Dropout Model Metrics", 
    X_vis=X_train, Y_vis=Y_train,
    df1_name="Uncalibrated", df2_name="Calibrated"
)


save_plot(
    plot_dual_expected_vs_empirical,
    save_dir="3D_Helmholtz", prefix="dropout_coverage",
    # loss=baseline_data_loss
)(
    dropout_pinn_df_uncal, dropout_pinn_df_cal,
    main_title="Dropout Model Coverage Plots"
)


plot_dual_expected_vs_empirical(
    dropout_pinn_df_uncal, dropout_pinn_df_cal,
    main_title="Dropout Model Coverage Plots"
)

In [ ]:
# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_metrics_table, plot_dual_expected_vs_empirical
save_plot(
    plot_metrics_table,
    save_dir="3D_Helmholtz", prefix="dropout",
    # loss=baseline_data_loss
    )(
    grid_test, dropout_pinn_uncal_predset, dropout_pinn_cp_cal_predset, 
    true_solution, 
    dropout_pinn_df_uncal, dropout_pinn_df_cal,
    title="2D UQ", main_title="Dropout Model Metrics", 
    X_vis=X_train, Y_vis=Y_train,
    df1_name="Uncalibrated", df2_name="Calibrated"
)


save_plot(
    plot_dual_expected_vs_empirical,
    save_dir="3D_Helmholtz", prefix="dropout_coverage",
    # loss=baseline_data_loss
)(
    dropout_pinn_df_uncal, dropout_pinn_df_cal,
    main_title="Dropout Model Coverage Plots"
)

In [ ]:
save_plot(
    plot_metrics_table,
    save_dir="3D_Helmholtz", prefix="dropout",
    # loss=baseline_data_loss
)(
    grid_test, dropout_pinn_uncal_predset, dropout_pinn_cp_cal_predset, 
    true_solution, 
    dropout_pinn_df_uncal, dropout_pinn_df_cal,
    title="2D UQ", main_title="Dropout Model Metrics", 
    X_vis=X_train, Y_vis=Y_train,
    df1_name="Uncalibrated", df2_name="Calibrated"
)


# save_plot(
#     plot_dual_expected_vs_empirical,
#     save_dir="3D_Helmholtz", prefix="dropout_coverage",
#     loss=baseline_data_loss
# )(
#     dropout_pinn_df_uncal, dropout_pinn_df_cal,
#     main_title="Dropout Model Coverage Plots"
# )

save_plot(
    plot_coverage_comparison,
    save_dir="3D_Helmholtz", prefix="dropout_coverage",
    # loss=baseline_data_loss
)(
    dropout_pinn_df_uncal, dropout_pinn_df_cal,
)

In [ ]:
plot_metrics_table(
    grid_test, dropout_pinn_uncal_predset, dropout_pinn_cp_cal_predset, 
    true_solution, 
    dropout_pinn_df_uncal, dropout_pinn_df_cal,
    title="2D UQ", main_title="Dropout Model Metrics", 
    X_vis=X_train, Y_vis=Y_train,
    df1_name="Uncalibrated", df2_name="Calibrated"
)

# Feature Distance Model

## Define Model

In [ ]:
from utils_uqmd.utils_uq_distance import DistanceUQPINN

# Base-Model Instance
model_args = dict(
    pde_class=pde, 
    input_dim=3,
    hidden_dims=[32, 64, 128, 128, 128, 64, 32], 
    output_dim=1
)

# CP-Model
cp_testing_args = {
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test, 
    "X_cal":X_calibration, "Y_cal":Y_calibration, 
    "X_train":X_train, "Y_train":Y_train, 
    "heuristic_u":"feature", # Change base on if the baseline cp
    "k":100
}

baseline_testing_args = { 
    # "uqmodel":dist_pinn
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test,
    "heuristic_u":"feature",
    "n_samples":15, 
}

dist_feat_pinn = DistanceUQPINN(**model_args)

## Training Base Model

In [ ]:
# Base Model
# Training
fit_args = dict(
    coloc_pt_num=colloc_pt_num, 
    X_train=X_train, 
    Y_train=Y_train
)
fit_kwargs_grid = dict(
    epochs=5000,
    λ_pde=1.0, λ_bc=5.0, λ_data=1.0,
    lr=1e-3, stop_schedule=20000
)
baseline_loss_dict = dist_feat_pinn.fit(**fit_args, **fit_kwargs_grid)

# Test the model performance
baseline_data_loss = dist_feat_pinn.data_loss(X_validation, Y_validation)


# Predicting
baseline_pred_kwargs = dict(
    n_samples=15, 
    heuristic_u="feature"
)
dist_feat_pinn_uncal_predset = dist_feat_pinn.predict(alpha, grid_test, 
                                   **baseline_pred_kwargs)


In [ ]:
dist_feat_pinn_uncal_predset = dist_feat_pinn.predict(alpha, grid_test, 
                                   **baseline_pred_kwargs)

## Create CP Model

In [ ]:
# CP Model
dist_feat_pinn_cp = CP(dist_feat_pinn)

# Predicting
cp_pred_kwargs = {
        "X_train":X_train,  "Y_train":Y_train,
        "X_cal":X_calibration, "Y_cal":Y_calibration,
        "heuristic_u":"feature",  # Change this based on cp
        "k":100
}

dist_feat_pinn_cp_cal_predset = dist_feat_pinn_cp.predict(
        alpha=alpha, X_test=grid_test,
        **cp_pred_kwargs
)

## Plot Coverage Plot

In [ ]:
# Computing Coverage Info
from utils_tools.utils_result_metrics import cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties
dist_feat_pinn_df_uncal = dist_test_uncertainties(uqmodel=dist_feat_pinn, **baseline_testing_args)
dist_feat_pinndf_cal = cp_test_uncertainties(dist_feat_pinn_cp, **cp_testing_args)

# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_metrics_table, plot_dual_expected_vs_empirical

In [ ]:
def plot_coverage_comparison(
    df_uncal,
    df_cal,
    *,
    labels=("Before CP", "After CP"),
    title="Coverage Comparison",
    ax=None,
    figsize=(5.8, 5),  # change this to control figure size when ax is None
):
    """Minimal coverage comparison plot."""
    import numpy as np
    import matplotlib.pyplot as plt

    def _prep(df):
        exp = 1.0 - df["alpha"].to_numpy()
        emp = df["coverage"].to_numpy()
        exp = np.concatenate(([0.0], exp, [1.0]))
        emp = np.concatenate(([0.0], emp, [1.0]))
        i = np.argsort(exp)
        return exp[i], emp[i]

    exp1, emp1 = _prep(df_uncal)
    exp2, emp2 = _prep(df_cal)

    if ax is None:
        fig, ax = plt.subplots(figsize=figsize)
    else:
        fig = ax.figure  # use existing figure/axes

    # curves + ideal
    ax.plot(exp1, emp1, 'o-', color="#b13c32", markerfacecolor="#ed8076", ms=11, lw=5,label=labels[0])
    ax.plot(exp2, emp2, '*-', color="#3245b1", markerfacecolor="#6988ef", ms=17, lw=5, label=labels[1])
    ax.plot([0, 1], [0, 1], '--', color='gray', label='Ideal (y=x)')

    # limits & labels
    ax.autoscale(enable=True, tight=False)
    ax.margins(x=0.02, y=0.02)      # tiny padding so endpoints aren’t cramped
    ax.set_xlabel("Expected Coverage (1 − α)")
    ax.set_ylabel("Empirical Coverage")
    # ax.set_title(title)
    ax.legend(loc='lower right', fontsize=15)
    fig.tight_layout()
    return ax


In [ ]:
save_plot(
    plot_metrics_table,
    save_dir="3D_Helmholtz", prefix="feature_distance",
    loss=baseline_data_loss
)(
    grid_test, dist_feat_pinn_uncal_predset, dist_feat_pinn_cp_cal_predset, 
    true_solution, 
    dist_feat_pinn_df_uncal, dist_feat_pinndf_cal,
    title="3D UQ", main_title="Feature Distance Model Metrics", 
    X_vis=X_train, Y_vis=Y_train,
    df1_name="Uncalibrated", df2_name="Calibrated",
)



save_plot(
    plot_coverage_comparison,
    save_dir="3D_Helmholtz", prefix="feature_distance_coverage",
    loss=baseline_data_loss
)(
    dist_feat_pinn_df_uncal, dist_feat_pinndf_cal, 
)

# Latent Distance Model

## Define Model

In [ ]:
from utils_uqmd.utils_uq_distance import DistanceUQPINN

# Base-Model Instance
model_args = dict(
    pde_class=pde, 
    input_dim=3,
    hidden_dims=[32, 64, 128, 128, 128, 64, 32], 
    output_dim=1
)

baseline_pred_kwargs = dict(n_samples=200)


# CP-Model

cp_testing_args = {
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test, 
    "X_cal":X_calibration, "Y_cal":Y_calibration, 
    "X_train":X_train, "Y_train":Y_train, 
    "heuristic_u":"latent", # Change base on if the baseline cp
    "k":100
}

baseline_testing_args = { 
    # "uqmodel":dist_pinn
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test,
    "heuristic_u":"latent",
    "n_samples":10, 
}

dist_lat_pinn = DistanceUQPINN(**model_args)

## Training Base Model

In [ ]:
# Base Model
# Training
fit_args = dict(
    coloc_pt_num=colloc_pt_num, 
    X_train=X_train, 
    Y_train=Y_train
)
fit_kwargs_grid = dict(
    epochs=5000,
    λ_pde=1.0, λ_bc=5.0, λ_data=1.0,
    lr=1e-3, stop_schedule=20000
)
baseline_loss_dict = dist_lat_pinn.fit(**fit_args, **fit_kwargs_grid)

# Test the model performance
baseline_data_loss = dist_lat_pinn.data_loss(X_validation, Y_validation)


# Predicting
baseline_pred_kwargs = dict(
    n_samples=10, 
    heuristic_u="latent"
)
dist_lat_pinn_uncal_predset = dist_lat_pinn.predict(alpha, grid_test, 
                                   **baseline_pred_kwargs)


## Create CP Model

In [ ]:
# CP Model
dist_lat_pinn_cp = CP(dist_lat_pinn)

# Predicting
cp_pred_kwargs = {
        "X_train":X_train,  "Y_train":Y_train,
        "X_cal":X_calibration, "Y_cal":Y_calibration,
        "heuristic_u":"latent",  # Change this based on cp
        "k":100
}

dist_lat_pinn_cp_cal_predset = dist_lat_pinn_cp.predict(
        alpha=alpha, X_test=grid_test,
        **cp_pred_kwargs
)

## Plot Coverage Plot

In [ ]:
# Computing Coverage Info
from utils_tools.utils_result_metrics import cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties
dist_lat_pinn_df_uncal = dist_test_uncertainties(uqmodel=dist_lat_pinn, **baseline_testing_args)
dist_lat_pinn_df_cal = cp_test_uncertainties(dist_lat_pinn_cp, **cp_testing_args)

# Save the plot
from utils_tools.utils_tuning import save_plot
from utils_tools.utils_result_viz import plot_metrics_table, plot_dual_expected_vs_empirical


In [ ]:
save_plot(
    plot_metrics_table,
    save_dir="3D_Helmholtz", prefix="latent_distance",
    loss=baseline_data_loss
)(
    grid_test, dist_lat_pinn_uncal_predset, dist_lat_pinn_cp_cal_predset, 
    true_solution, 
    dist_lat_pinn_df_uncal, dist_lat_pinn_df_cal,
    title="2D UQ", main_title="Latent Distance Model Metrics", 
    X_vis=X_train, Y_vis=Y_train,
    df1_name="Uncalibrated", df2_name="Calibrated"
)



save_plot(
    plot_coverage_comparison,
    save_dir="3D_Helmholtz", prefix="lat_distance_cov",
    loss=baseline_data_loss
)(
    dist_lat_pinn_df_uncal, dist_lat_pinn_df_cal,
)

# VI Model

## Define Model

In [ ]:
# from utils_uqmd.utils_uq_vi import VIBPINN

# # Base Model Instance
# model_args = {
#     "pde_class":pde,
#     "input_dim":3,
#     "hidden_dims":[32, 64, 128, 264, 264, 264, 128, 64, 32], 
#     "output_dim":1,
#     "mu_std" : 0.01, "rho" : -6, "prior_std" : 1.0, 
#     "init_data_noise" : 1.0, "learn_data_noise" : False, 
#     "act_func" : nn.Tanh()
# }
# baseline_pred_kwargs = {
#     "n_samples":5000 
# }

# baseline_testing_args = { 
#     # "uqmodel":vi_model, 
#     "alphas":alphas, 
#     "X_test":X_test, "Y_test":Y_test
# }

# vi_bpinn = VIBPINN(**model_args)

## Training Base Model

In [ ]:
# # Base Model
# # Training
# fit_args = {
#     "coloc_pt_num":colloc_pt_num,
#     "X_train":X_train, 
#     "Y_train":Y_train
# }
# fit_kwargs_grid = {
#     "epochs":50000,
#     "λ_pde":1.0, "λ_bc":1.0, "λ_elbo":1.0,  
#     "lr":1e-3,
#     "stop_schedule":20000,
#     "scheduler_kwargs":{'step_size': 5500, 'gamma': 0.5}
# }
# baseline_loss_dict = vi_bpinn.fit(**fit_args, **fit_kwargs_grid)

# # Test the model performance
# baseline_data_loss = vi_bpinn.data_loss(X_validation, Y_validation)


# # Predicting
# baseline_pred_kwargs = dict(
#     n_samples=20
# )
# vi_bpinn_uncal_predset = vi_bpinn.predict(alpha, grid_test, 
#                                    **baseline_pred_kwargs)


## Create CP Model

In [ ]:
# # CP Model
# vi_bpinn_cp = CP(vi_bpinn)

# # CP
# cp_pred_kwargs = {
#         "X_train":X_train,  "Y_train":Y_train,
#         "X_cal":X_calibration, "Y_cal":Y_calibration,
#         "heuristic_u":"raw_std",
#         "k":10
# }

# cp_testing_args = {
#         "alphas":alphas, 
#         "X_test":X_test, "Y_test":Y_test, 
#         "X_cal":X_calibration, "Y_cal":Y_calibration, "X_train":X_train, "Y_train":Y_train, 
#         "heuristic_u":"raw_std", # Change base on if the baseline model has its original uq band
#         "k":10
# }

# vi_bpinn_cp_cal_predset = vi_bpinn_cp.predict(
#         alpha=alpha, X_test=grid_test,
#         **cp_pred_kwargs
# )

## Plot Coverage Plot

In [ ]:
# # Computing Coverage Info
# from utils_tools.utils_result_metrics import cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties
# vi_bpinn_df_uncal = vi_test_uncertainties(uqmodel=vi_bpinn, **baseline_testing_args)
# vi_bpinn_df_cal = cp_test_uncertainties(vi_bpinn_cp, **cp_testing_args)

# # Save the plot
# from utils_tools.utils_tuning import save_plot
# from utils_tools.utils_result_viz import plot_metrics_table
# save_plot(
#     plot_metrics_table,
#     save_dir="3D_Helmholtz", prefix="vi_bpinn",
#     loss=baseline_data_loss
# )(
#     grid_test, vi_bpinn_uncal_predset, vi_bpinn_cp_cal_predset, 
#     true_solution, 
#     vi_bpinn_df_uncal, vi_bpinn_df_cal,
#     title="2D UQ", main_title ="VI Model Metrics", 
#     X_vis=X_train, Y_vis=Y_train,
#     df1_name="Uncalibrated", df2_name="Calibrated"
# )

# from utils_tools.utils_result_viz import plot_dual_expected_vs_empirical
# save_plot(
#     plot_dual_expected_vs_empirical,
#     save_dir="3D_Helmholtz", prefix="vi_bpinn_coverage",
#     loss=baseline_data_loss
# )(
#     vi_bpinn_df_uncal, vi_bpinn_df_cal,
#     main_title = "VI BPINN Coverage Plots"
# )

In [ ]:
# save_plot(
#     plot_metrics_table,
#     save_dir="3D_Helmholtz", prefix="vi_bpinn",
#     loss=baseline_data_loss
# )(
#     grid_test, vi_bpinn_uncal_predset, vi_bpinn_cp_cal_predset, 
#     true_solution, 
#     vi_bpinn_df_uncal, vi_bpinn_df_cal,
#     title="2D UQ", main_title ="VI Model Metrics", 
#     X_vis=X_train, Y_vis=Y_train,
#     df1_name="Uncalibrated", df2_name="Calibrated"
# )

# # save_plot(
# #     plot_dual_expected_vs_empirical,
# #     save_dir="3D_Helmholtz", prefix="dropout_coverage",
# #     loss=baseline_data_loss
# # )(
# #     dropout_pinn_df_uncal, dropout_pinn_df_cal,
# #     main_title="Dropout Model Coverage Plots"
# # )

# save_plot(
#     plot_coverage_comparison,
#     save_dir="3D_Helmholtz", prefix="vi_bpinn_coverage",
#     loss=baseline_data_loss
# )(
#     vi_bpinn_df_uncal, vi_bpinn_df_cal,
# )

# HMC Model

## Define Model

In [ ]:
# from utils_uqmd.utils_uq_hmc import HMCBPINN

# # Base Model Instance
# model_args = dict(
#     pde_class=pde, input_dim=3, 
#     hidden_dims=[32, 64, 128, 264, 264, 264, 128, 64, 32],
#     output_dim=1, act_func=nn.Tanh, prior_std=1.0,
#     step_size=1e-3, leapfrog_steps=5
# )

# baseline_testing_args = {
#     # "uqmodel": hmc_model,
#     "alphas": alphas,
#     "X_test": X_test,
#     "Y_test": Y_test,
#     "n_samples": 5000
# }

# baseline_pred_kwargs = {"n_samples": 5000}

# hmc_bpinn = HMCBPINN(**model_args)

## Training Base Model

In [ ]:
# # Base Model
# # Training
# fit_args = {
#     "coloc_pt_num":colloc_pt_num,
#     "X_train":X_train, 
#     "Y_train":Y_train
# }
# fit_kwargs_grid = {
#     "λ_pde": 1.0,
#     "λ_bc": 3.0,
#     "λ_data": 5.0,
#     "epochs": 5000,
#     "lr":1e-3,
#     "hmc_samples": 15000,
#     "brun_in":5000,
#     "step_size": 1e-4,
#     "leapfrog_steps": 13
# }
# baseline_loss_dict = hmc_bpinn.fit(**fit_args, **fit_kwargs_grid)

# # Test the model performance
# baseline_data_loss = hmc_bpinn.data_loss(X_validation, Y_validation)

# hmc_bpinn_uncal_predset = hmc_bpinn.predict(
#             alpha=alpha, X_test=X_test,
#             **baseline_pred_kwargs
#         )

## Create CP Model

In [ ]:
# # CP Model
# hmc_bpinn_cp = CP(hmc_bpinn)

# # CP
# cp_pred_kwargs = { 
#     "X_train": X_train, "Y_train": Y_train,
#     "X_cal": X_calibration, "Y_cal": Y_calibration,
#     "heuristic_u": "raw_std", "k": 100
# }

# cp_testing_args = {
#     "alphas":alphas, 
#     "X_test":X_test, "Y_test":Y_test, 
#     "X_cal":X_calibration, "Y_cal":Y_calibration, "X_train":X_train, "Y_train":Y_train, 
#     "heuristic_u":"raw_std", # Change base on if the baseline model has its original uq band
#     "k":100
# }

# hmc_bpinn_cp_cal_predset = hmc_bpinn_cp.predict(
#         alpha=alpha, X_test=grid_test,
#         **cp_pred_kwargs
# )

## Plot Coverage Plot

In [ ]:
# # Computing Coverage Info
# from utils_tools.utils_result_metrics import hmc_test_uncertainties, cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties, dist_test_uncertainties
# hmc_bpinn_df_uncal = hmc_test_uncertainties(uqmodel=hmc_bpinn, **baseline_testing_args)
# hmc_bpinn_df_cal = cp_test_uncertainties(hmc_bpinn_cp, **cp_testing_args)

# # Save the plot
# from utils_tools.utils_tuning import save_plot
# from utils_tools.utils_result_viz import plot_metrics_table, plot_dual_expected_vs_empirical
# save_plot(
#     plot_metrics_table,
#     save_dir="3D_Helmholtz", prefix="hmc_bpinn",
#     loss=baseline_data_loss
# )(
#     grid_test, hmc_bpinn_uncal_predset, hmc_bpinn_cp_cal_predset, 
#     true_solution, 
#     hmc_bpinn_df_uncal, hmc_bpinn_df_cal,
#     title="2D UQ", main_title="HMC Model Metrics", 
#     X_vis=X_train, Y_vis=Y_train,
#     df1_name="Uncalibrated", df2_name="Calibrated"
# )

# save_plot(
#     plot_dual_expected_vs_empirical,
#     save_dir="3D_Helmholtz", prefix="hmc_bpinn_coverage",
#     loss=baseline_data_loss
# )(
#     hmc_bpinn_df_uncal, hmc_bpinn_df_cal,
#     main_title="HMC BPINN Coverage Plots"
# )

In [ ]:

# save_plot(
#     plot_coverage_comparison,
#     save_dir="3D_Helmholtz", prefix="hmc_bpinn_coverage",
#     loss=baseline_data_loss
# )(
#     hmc_bpinn_df_uncal, hmc_bpinn_df_cal,
# )

# Coverage Plotting

In [ ]:
def plot_coverage_grid_2x2(
    named_pairs,
    *,
    labels=("Before CP", "After CP"),
    grid_figsize=(14.9, 5),
    suptitle=None,
    model_title_size=19,
):
    """
    named_pairs: list or dict of exactly 4 items like:
        [("Model A", df_uncal_A, df_cal_A), ...]
        OR {"Model A": (df_uncal_A, df_cal_A), ...}
    """
    import matplotlib.pyplot as plt
    import numpy as np

    # --- same prep as in plot_coverage_comparison ---
    def _prep(df):
        exp = 1.0 - df["alpha"].to_numpy()
        emp = df["coverage"].to_numpy()
        exp = np.concatenate(([0.0], exp, [1.0]))
        emp = np.concatenate(([0.0], emp, [1.0]))
        i = np.argsort(exp)
        return exp[i], emp[i]

    # normalize input
    items = list(named_pairs.items()) if isinstance(named_pairs, dict) else list(named_pairs)
    # if len(items) != 4:
        # raise ValueError("Provide exactly 4 (name, df_uncal, df_cal) pairs.")

    fig, axs = plt.subplots(1, 3, figsize=grid_figsize)

    i = 0
    for ax, (name, df_uncal, df_cal) in zip(axs.ravel(), items):
        # --- begin: copied body of plot_coverage_comparison (minimal version) ---
        exp1, emp1 = _prep(df_uncal)
        exp2, emp2 = _prep(df_cal)

        # curves + ideal
        ax.plot(exp1, emp1, 'o-', color="#3245b1", markerfacecolor="#6988ef", ms=11, lw=4, label=labels[0])
        ax.plot(exp2, emp2, '*-', color="#b13c32", markerfacecolor="#ed8076", ms=17, lw=4, label=labels[1])
        ax.plot([0, 1], [0, 1], '--', color='gray', label='Ideal (y=x)')
        i+=1
        if i <= 1:
            # limits & labels
            ax.autoscale(enable=True, tight=False)
            ax.margins(x=0.03, y=0.03)      # tiny padding so endpoints aren’t cramped
            ax.set_xlabel("Expected Coverage (1 − α)", fontsize=16)
            ax.set_ylabel("Empirical Coverage", fontsize=16)
            ax.legend(loc='lower right', fontsize=14.5)
            # --- end: copied body ---

            # subplot title = model name
            ax.set_title(name, fontsize=model_title_size)
        else:
            ax.autoscale(enable=True, tight=False)
            ax.margins(x=0.03, y=0.03)      # tiny padding so endpoints aren’t cramped
            # ax.legend(loc='upper left', fontsize=13)
            ax.set_xlabel("Expected Coverage (1 − α)", fontsize=16)
            # --- end: copied body ---
            ax.yaxis.set_visible(False)
            # subplot title = model name
            ax.set_title(name, fontsize=model_title_size)

    if suptitle:
        fig.suptitle(suptitle, fontsize=model_title_size + 2)
        fig.tight_layout(rect=[0, 0, 1, 0.96])
    else:
        fig.tight_layout()

    return fig, axs


In [ ]:
pairs = [
    ("GD",     dist_feat_pinn_df_uncal, dist_feat_pinndf_cal),
    ("LD",    dist_lat_pinn_df_uncal, dist_lat_pinn_df_cal),
    ("Dropout", dropout_pinn_df_uncal, dropout_pinn_df_cal),
]
fig, axs = plot_coverage_grid_2x2(pairs)




save_plot(

    plot_coverage_grid_2x2,
    save_dir="3D_Helmholtz", prefix="coverage_sets",
    loss=baseline_data_loss
)(pairs)

In [ ]:
def plot_coverage_grid_1x2(
    named_pairs,
    *,
    labels=("After CP", "After Local CP"),
    grid_figsize=(16, 7.5),
    suptitle=None,
    model_title_size=22,
    middle_ratio=0.55,              # width of legend column relative to each plot
):
    """
    named_pairs: list or dict of exactly 2 items like:
        [("Model A", df_uncal_A, df_cal_A), ("Model B", df_uncal_B, df_cal_B)]
        OR {"Model A": (df_uncal_A, df_cal_A), "Model B": (df_uncal_B, df_cal_B)}
    """
    import matplotlib.pyplot as plt
    import numpy as np

    def _prep(df):
        exp = 1.0 - df["alpha"].to_numpy()
        emp = df["coverage"].to_numpy()
        exp = np.concatenate(([0.0], exp, [1.0]))
        emp = np.concatenate(([0.0], emp, [1.0]))
        i = np.argsort(exp)
        return exp[i], emp[i]

    # normalize input
    items = list(named_pairs.items()) if isinstance(named_pairs, dict) else list(named_pairs)
    if len(items) != 2:
        raise ValueError("Provide exactly 2 (name, df_uncal, df_cal) pairs.")

    # 1x3 layout: [plot1 | legend | plot2]
    fig, axs = plt.subplots(
        1, 3,
        figsize=grid_figsize,
        sharey=True,
        gridspec_kw={"width_ratios": [1.0, middle_ratio, 1.0]}
    )

    legend_handles, legend_labels = None, None

    for ax_plot, (name, df_uncal, df_cal) in zip([axs[0], axs[2]], items):
        exp1, emp1 = _prep(df_uncal)
        exp2, emp2 = _prep(df_cal)

        ax_plot.plot(exp1, emp1, 'o-', color="#b13c32", markerfacecolor="#ed8076",
                     ms=14, lw=5, label=labels[0])
        ax_plot.plot(exp2, emp2, '*-', color="#3291b1", markerfacecolor="#69b5ef",
                     ms=20, lw=5, label=labels[1])
        ax_plot.plot([0, 1], [0, 1], '--', color='gray', label='Ideal (y=x)')

        # collect legend once
        if legend_handles is None:
            legend_handles, legend_labels = ax_plot.get_legend_handles_labels()

        # axis cosmetics
        ax_plot.set_xlabel("Expected Coverage (1 − α)", fontsize=18)
        ax_plot.autoscale(enable=True, tight=False)
        ax_plot.margins(x=0.02, y=0.02)
        ax_plot.set_title(name, fontsize=model_title_size)

    # Y label only on the left plot
    axs[0].set_ylabel("Empirical Coverage", fontsize=18)

    # Middle axis: legend only
    axs[1].axis("off")
    for spine in axs[1].spines.values():
        spine.set_visible(False)
    # Put the legend centered in the middle axis
    axs[1].legend(
        legend_handles, legend_labels,
        loc="center",
        frameon=False,
        ncol=3,
        fontsize=17
    )

    if suptitle:
        fig.suptitle(suptitle, fontsize=model_title_size + 2, y=0.98)

    fig.tight_layout(rect=[0, 0, 1, 0.96])

    return fig, axs


In [ ]:
pairs = [
    ("GD",     dist_feat_pinn_df_uncal, dist_feat_pinndf_cal),
    ("LD",    dist_lat_pinn_df_uncal, dist_lat_pinn_df_cal),
]
fig, axs = plot_coverage_grid_1x2(pairs)



In [ ]:
pairs = [
    ("GD",     dist_feat_pinn_df_uncal, dist_feat_pinndf_cal),
    ("LD",    dist_lat_pinn_df_uncal, dist_lat_pinn_df_cal),
]
fig, axs = plot_coverage_grid_1x2_centered_legend(pairs)

